In [20]:
import pandas as pd
import numpy as np
import time
from twython import Twython, TwythonError, TwythonRateLimitError
import json 
import sklearn
from sklearn.feature_extraction import text
import pandas_profiling
from pandas_profiling.utils.cache import cache_file 

In [21]:
#Set up API access information
APP_KEY = 'SECRET CODE'
APP_SECRET = 'SECRET CODE'

twitter = Twython(APP_KEY, APP_SECRET, oauth_version=2)
ACCESS_TOKEN = twitter.obtain_access_token()

In [22]:
#Use Twython to connect to twitter
twitter = Twython(APP_KEY, access_token=ACCESS_TOKEN)

In [103]:
#Search twitter for set hashtags-- initial collection is for only 6 hashtags
query = '#gsmnp OR #GrandCanyon OR #rmnp OR #ZionNPS OR #Yosemite OR #NPS'

In [ ]:
listed = pd.DataFrame()
request_pages = 450
for i in range(0, request_pages):
    if i == 0: #on intial request, search for query
        q_tweet = twitter.search(q = query, count = '100', lang = 'en')
    else: #for additional request, search query and use the previous minimum id value as starting max value
        q_tweet = twitter.search(q = query, count = '100', lang = 'en',
                                max_id = curr_since)
    t_df = pd.json_normalize(q_tweet['statuses']) #normalize data
    if len(t_df.index) == 0: #if the length of the dataframe is equal to 0, break
        break
    curr_since = min(t_df['id']) #initial minimum id
    curr_since = str(curr_since) #convert to string
    listed = listed.append(t_df) 
    time.sleep(2)

In [192]:
#Save the data to .csv
listed.to_csv('data_df.csv', index=False) #save test file so not continually pulling from Twitter

In [194]:
#Open file
df = pd.read_csv('data_df.csv', low_memory= False)
df.describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Columns: 348 entries, created_at to quoted_status.place.bounding_box.coordinates
dtypes: bool(15), float64(99), int64(11), object(223)
memory usage: 5.6+ MB


In [196]:
print(df.columns)

Index(['created_at', 'id', 'id_str', 'text', 'truncated', 'source',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       ...
       'quoted_status.place.id', 'quoted_status.place.url',
       'quoted_status.place.place_type', 'quoted_status.place.name',
       'quoted_status.place.full_name', 'quoted_status.place.country_code',
       'quoted_status.place.country', 'quoted_status.place.contained_within',
       'quoted_status.place.bounding_box.type',
       'quoted_status.place.bounding_box.coordinates'],
      dtype='object', length=348)


In [197]:
#Check to see if any null values in text column
print(df['text'].isnull().values.any())

False
